<a href="https://colab.research.google.com/github/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/chap03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第三章 リカレントニューラルネットワークと自然言語処理

本章では、自然言語処理で主に使われるRNN（リカーレントニューラルネットワーク）の基礎からより応用的で現在最新の研究に近いアテンションやBERTに関する理解を目指して説明していきます。


## 3.1 はじめに

本章では自然言語の話をしていきます。自然言語とは、人と人との日常の意思疎通に用いられる自然発生的な言語のことです。簡単にいうと、日本語や英語のような一般に用いられる言語のことを指します。対照的に、プログラミング言語など、ルールが厳格に決められた言語は形式言語などと呼ばれます。

前章では画像認識などといったコンピュータービジョンの分野を扱いました。コンピュータービジョンと自然言語処理は何か違うのでしょうか。コンピュータービジョンというのは、コンピューターに目のような機能を持たせる技術と言えます。コンピュータービジョンでは、例えば画像を見て何が写っているかを判断する、というような人間の目が行うことを行います。言い換えると、解析対象は物理的な光です。このような解析は人間だけができるものではなくて、犬とか猫などにもできますし、より小さな動物でも可能なタスクです。もちろん、ある程度複雑なタスクではありますが、ディープラーニングの技術が発達した今となってはそこまで難しすぎるタスクではないと言ってもいいかもしれません。

それに対して、自然言語は基本的に人間しか認知できません。たとえば、下図の左の鳥の画像のような「実態」に対して、人間が認知するための次元の異なる概念としての「鳥」という言葉を当てているわけです。また、左の赤い丸のような実態を見て、これは「りんご」であると認知するわけです。

![chap03_cv_vs_nlp](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_cv_vs_nlp.png?raw=true)
<center>図　コンピュータービジョンと自然言語処理</center>

簡単に、コンピュータービジョンや自然言語処理では、そもそもの扱う対象が異なるというところを認識していただければと思います。

さて、目で見た物体が持つ特徴というのは、例えば「丸みがある」だとか「角張っている」だとか「赤い」など、形や色などに関連するものであり、直感的にイメージしやすいでしょう。

では自然言語で構成される文章が持つ特徴とはどんなものでしょうか。本の文章から著者を推定するようなタスクであれば、よく使う単語があるとか、一文の長さが長いとか、比喩表現を多用するとかいったところに著者のクセ、つまり特徴が現れるかもしれません。この場合は特徴量として単語の出現回数や文章の長さなどを用いると良さそうです。

質問応答のようなタスクではどうでしょう。適切な回答を返すためにはまず質問の内容そのものを理解する必要があります。会話の中で出てきた質問であれば文脈を記憶しておくことも必要そうです。ImageNetデータセットで訓練したニューラルネットワークがコンピュータービジョンの幅広いタスクに対する特徴抽出器として用いられるように、自然言語処理のタスクに対して汎用的に利用できる特徴抽出器はないのでしょうか。



2012年に畳み込みニューラルネットワークのAlexNetがImageNetコンペティションで優勝してから6年の月日が経ち、2018年にBERTという新しいモデルがGoogleによって発表されました。まるでImageNetで事前訓練したモデルのように、Wikipediaなどの大規模なコーパスを利用してBERTを事前訓練することで、従来のモデルでは難しかった様々なタスクに対して好成績を修めました。この衝撃的な結果は研究領域内のみならず、SNSやメディアなどでも大きく取り上げられました。

BERTとは "Bidirectional Encoder Representation from Transformers" の頭文字をとったものです。これを日本語に意訳すると「双方向のTransformerを用いて言語表現をエンコードするモデル」のようになりますが、日本語にしても一見して理解することは難しいのではないかと思います。

まず、このBERTの核をなす「双方向のTransformer」の「Transformer」が何かというと、これはGoogleが開発したニューラル機械翻訳モデルの一種です。機械翻訳には以前はRNN（リカレントニューラルネットワーク、特にLSTMなど）がよく用いられていたのですが、TransformerではRNNを使わずにAttention機構が利用されています。

本節では、BERTの雰囲気を理解していただくことを目標として、自然言語処理とRNN、Attention機構とは何か、Transformer/BERTの概要という内容について3本立てで説明をしていきます。

## 3.2 自然言語処理とは

まず、機械学習を用いた自然言語処理の話をします。先述の通り、自然言語処理とは人間が使う言葉を機械で処理をするというものです。自然言語処理の応用のタスクにどのようなものとして下図のようなものがあります。

![chap03_cv_vs_nlp](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_nlp_applications.png?raw=true)
<center>図　自然言語処理の主要な応用タスク</center>

たとえばわかりやすいところで言うと、ニュース記事があって、その内容が科学技術について書かれたものなのか、スポーツのニュースなのか、ゴシップ系の芸能ニュースであるのかなどを分類したいとします。人手でニュース記事の分類を行う場合にはざっと文章を眺めていくつかの特徴的な単語を見つければ簡単に分類できます。「野球」と「試合」という単語がニュース記事内にあればそれは高い確率でスポーツニュースでしょう。こう言ったことを機械で行うタスクを文章分類と呼びます。

もっと馴染み深いところで言うとGoogle翻訳やDeepLなどが行う日英翻訳などの機械翻訳も自然言語処理のタスクのひとつです。

また、大量の文章を短い文章にまとめる文章要約というタスクもあります。2019年には、Element AIのメンバーらにより公開された文章要約に関する論文 ([On Extractive and Abstractive Neural Document Summarization with Transformer Language Models](https://arxiv.org/abs/1909.03186)) が話題になりました。というのも、論文にはかならず最初に記述されているアブストラクト（要旨）がありますが、その論文のアブストラクト自体を論文で用いられている文章要約の技術を用いて機械的に生成されていたのです。実際その要約は人間が書いたような自然な文章で書かれています。

あとはチャットボットに代表されるような質問応答などの応用タスクがあります。

## 3.3 リカレントニューラルネットワーク

このようなタスクはニューラルネットワークによって解くのが近年のトレンドとなっています。ただし、一口にニューラルネットワークといっても実際はどういった構造のニューラルネットワークを用いれば良いかは自明ではありません。

一般的に、自然言語処理のタスクを普通の全結合ニューラルネットワークで解こうとするとなかなか難しいことが多いです。なぜ難しいのでしょうか。

例えば、英語の文章を日本語に翻訳するタスクを解きたいとしましょう。まず、英語の文章をニューラルネットワークに対して入力します。たとえば下図のように、"Dogs are cute." という文章を入力したら「犬はかわいい。」というような文章が出てきてほしいというような問題設定でニューラルネットワークを訓練していきます。

一般に入力部の長さや出力部の長さはセンテンスごとに異なります。今回の例では、"Dogs are cute." という3つの単語（ピリオドも単語に含めると定義すれば4単語）を入力としていますが、もっと短い文章であったり長い文章であったりすることもあります。そのため、全結合ニューラルネットワークのように固定の長さの入力があって中間層があって固定の長さの出力層があるというような全結合ニューラルネットワークだとなかなか扱いづらいです。

たとえば100個の入力のユニットを用意しておいて、今回の例のように"Dogs are cute." という3つの単語を入力してあとはブランクであるという情報を入れるような仕組みを利用することもできますが、ブランクが多すぎて学習が進みづらいとか、そもそも200個の単語があった場合にはどうなるかというようにどんどん対応ができなくなるため、このようなアプローチは単純にはうまくいかないでしょう。（ただし、後述するBERTなどのモデルでは固定数の入出力ユニットを用いています。）

![chap03_dense_net_nlp_challenge](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_dense_net_nlp_challenge.png?raw=true)
<center>図　全結合ニューラルネットワークの課題</center>

またこの例では、1番目の入力に "dog" という単語があるとそれは「犬」に対応する、といったことがひとまず学習されるでしょう。しかし、"I like the dog." というような別の文章が入力されたとき、"dog" をどう扱って翻訳すればよいかは自明ではありません。全結合ニューラルネットワークでは、同じ単語でも入力位置によって別物として扱われてしまうため、一回学んだことが次に応用できなくなってしまいます。文章は刻一刻と単語が変化するシーケンス（時系列データ）と考えると、時系列データに適したニューラルネットワークを考える必要がありそうです。

そこでよく用いられるのがリカレントニューラルネットワーク (Recurrent Neural Network; RNN) です。再帰型ニューラルネットワークなどと呼ばれることもあります。時系列の情報を持つデータ、つまり順番によって内容が変わるようなデータというものに対して広く用いられています。

### 3.3.1 リカレントニューラルネットワークの仕組み

リカレントニューラルネットワークとはどういう仕組みで動いているのかについてもう少し詳しく説明していきます。

下図はひとつの全結合ニューラルネットワークだと思ってください。この図では下方から上方に処理が進んでいきます。まず、一番下からの入力を $x^{<1>}$ とします。この文脈では $x^{<t>}\,(t=0,...,T)$ はひとつの単語と考えてください。

（注：文字列そのままではニューラルネットワークに入力できないので、なんらかの数値やベクトルなどに変換する必要があります。このトピックに関しては後述します。）

入力に対して重み $W_x$ をかけて、中間層の各列の状態 $a^{<1>}$ を得ます。そしてまた出力側の重み $W_y$ をかけて、出力 $y^{<1>}$ を得るといった一連の処理を行っています。

先ほどの翻訳の例で言うと、"Dogs are cute." の最初の単語 "Dogs" が $x^{<1>}$ に対応します。それを変換し、出力 $y^{<1>}$ は「犬」となることを期待するのです。

![chap03_rnn_1](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_1.png?raw=true)

上記の処理を数式で書くと以下のようになります。ただし、入力層側の活性化関数とバイアスはそれぞれ $f$ と $b_x$ 、出力層側の活性化関数とバイアスをそれぞれ $g$ と $b_y$ としています。

$$ a^{<1>}=f(W_x \cdot x^{<1>}+b_x)$$
$$ y^{<1>}=g(W_y \cdot a^{<1>}+b_y)$$

もちろんこれだけで終わるのではなく、次の時刻には新しい入力が入ってくるでしょう。その場合に上記の処理を繰り返すのでは全結合ニューラルネットワークのときと変わりません。リカレントニューラルネットワークでは下図のように、その時点における入力 $x^{<2>}$ だけではなく、その1個前の中間状態 $a^{<1>}$ も重み $W_a$ をかけて加算します。

引き続き翻訳で例えると、"Dogs are cute." の "are" がここでの $x^{<2>}$ に対応します。"are" だけを見て、"are" は日本語に直すと「は」である、という推論をするのではなく、"Dogs are" を見て、「犬」に続く単語は「は」であると推論していることになります。

![chap03_rnn_2](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_2.png?raw=true)

数式で書くと以下のようになります。

$$ a^{<2>}=f(W_a \cdot a^{<1>} + W_x \cdot x^{<2>} + b_x)$$
$$ y^{<2>}=g(W_y \cdot a^{<2>}+b_y)$$

それをさらに下図のように繰り返していくわけです。

![chap03_rnn_3](https://github.com/kmotohas/shuwa-book-dl4prac-examples/blob/master/notebooks/figures/chap03_rnn_3.png?raw=true)

<<< ここまでチェック >>>

このようにして、今までの時系列データの情報をメモリのようなものを作って中間の状態にどんどん貯めていって、今までの入力の状態を考慮しながら次の出力を得ていくという手法がリカーレントニューラルネットワーク（RNN）で行っていることとなります。

上図のネットワークは省略して、下図の右の赤枠のように記述することがあります。リカーレントネットワークは、自分の中間の状態が次また入ってくるというようなループがあるので、そのループが再帰的であることから、再帰的（リカーレント）という呼び方がされているネットワークです。

このようなネットワークを構築すると何がうれしいかというと、すでに少しお話しましたが、下図のように、さきほどの例の’Dogs are cute’のような文章があった場合、cuteという単語を入力したときに、かわいいと出てきて欲しいのですが、リカーレントネットワークを用いるとdogs areというような前のデータの状態も考慮して最後のかわいいという答えを出すようなことが可能になります。つまり今までの入力のデータをあいまいで抽象的な状態にエンコードして出力を出すということができるようになっています。

RNNのユニット

それでは、RNNのユニットの詳細についてこれから見ていきます。
RNNの１つ１つのユニットはさきほどの図でいうと、４つの〇で構成される真ん中のところとなり、これを１つのユニットだと思って回路のように書き下出してみると、下図のようになります。


RNNのユニットの動作は、上図の現時点の入力のx<t>とその前の時点の中間層の状態のx<t-1>がtanhの数式で示される行列で掛け算されて活性化関数の出力を次に出すということを延々と繰り返すというものです。そこで何か出力を出したい場合に、たとえば確率的な出力を出したい場合には、上図の例のようなSoftmaxと呼ばれる活性化関数にtanh関数の計算結果を入力して、Softmaxを用いて一番確率が多いものを出力として出すようなことを行います。しかしながらこのユニットは実は簡略化されすぎています。たとえば、次の図にあるような’The cat which already ate …, [was] full’のような‘猫がいろんなものをたくさん食べた結果おなかがいっぱいです’というような文章を出力したいときに、図の[]内のbe動詞に何が入るかを予測するという課題では、今回の例では、ateという単語が前にあるので、過去の状態を表しているという条件に引きずられてwasになるというようなロジックが必要になります。ただしcatが２番目の文章のようにcatsという複数形になった場合には、さらにwereになるわけです。このような文章の場合、catとfullの間に挟まれているほかの文章は関係がなく、本文の冒頭の名詞であるcatが単数か複数化にだけに依存します。そのため、[]内の正しいbe動詞を得たいときに、中間の文章は気にしないというような操作もニューラルネットワークの機能の中に入っていて欲しいという思想が生まれ、それを実現するためのゲートが追加されたGRUと呼ばれるRNNの進化版にあたるものが提唱されています。

GRUが何かというと、今までのRNNは入力と前の中間層の状態にバイアスをプラスして活性化関数とかけたものが出力だったわけですが、それに図の右の四角のようにsigmoid関数と呼ばれる０～１の出力値を返す活性化関数が追加されています。このsigmoid関数に対しても入力と一つ前の中間層の状態を入れることで、RNNに0か１つまりTrueがFalseにあたるスイッチのオンオフ機能をつけることができるようになります。これをさきほどの通常のRNNで得られる図の左側の <t>に掛け算してあげれば、たとえば、これが１を返せば、中間層の をそのまま出力し、０であれば、次の状態に渡さないので、その前のa<t-1>の状態がそのまま出力されるというような形の情報の流れにすることができます。イメージとしては、catが入ってきて、whichとかalreadyとかate以外はすべて0を掛けて、必要となるものだけを覚えて、[]の出力の計算にその出力を使うということができるというようなものが学習できるような機構が追加されたものです。

GRUというのは2014年に提唱されたモデルですけど、実際には図にあるアップデートデートという情報を更新するか否かを決めるような0か1かだけでなくて、リセットゲートと呼ばれるそもそも情報を忘れる、つまり今までのメモリをリセットするという別の計算もあります。ただしこれらは概念的には同じような機能となります。RNNの仲間にはGRUの他に、LSTMと呼ばれるGRUよりもさらに有名なモデルがありますが、これもゲートの種類と数が違うだけで、ほとんど似たような構造で少し複雑ではありますがGRUと同じような考え方で理解できるものなので、式を順に追っていけば理解可能なそんなに難しい概念ではありません。

次に機械翻訳という文脈の中では、１つ強力なもので下図に示すようなseq2seqというモデルが存在します。これはシーケンス・ツー・シーケンスの略で、エンコーダ-デコーダモデルとも呼ばれています。これはどのようなイメージかと言いますと、さきほどのようなイメージと同じで、下図の四角にそれぞれRNN、GRU、LSTMなどのユニットがあって、それに対して文章を下からどんどん入力していきます。そこでエンコードされた図で示すところのCにあたる場所で文脈（コンテキスト）のような状態を次のRNN等に対してまた入力してあげて、新しい出力を得るというようなことを行っています。イメージとしては’Dogs are cute.’というような文章があったときに、それを図のCの場所で中間的な状態に押し込んで、犬はかわいいという文章を出すというような、一度文を抽象的な状態にエンコードして、それをまた日本語の文章としてデコードするというようなことを行っているモデルです。

上記のモデルは今でもよく使われているものではありますが、入力文が長くなりすぎる場合、たとえば英語の長文などをこのCの場所でそれまでの状態を押し込める、つまりは大量の情報を一つのベクトル詰め込んでしまう結果、学習効率があまり良くはありません。それを解決するためにAttention（注意）機構が導入されました。

Attention機構は非常に簡単な概念で、イメージで言いますと、例えば’Dogs are cuter than cats.’みたいな文章があったときに、これを日本語に翻訳したいとします。結果として’犬は猫よりかわいい(という感じの文章になれば良いわけですが、犬という単語を出力したいときに、あまり後半の単語は気にせず’犬は’という部分に注力して見ていれば最初に出すべき日本語は犬だということが言えるわけです。つまりある程度どこに注目すべきかを重みをつけてあげるという機構がAttentionと呼ばれるものです。たとえば、次の単語の’は’を出したい場合には、’are’という単語を見れば’は’という単語が出てくるとか、下図のよう’cats’のところを見ておけば’猫’で、’than’とかがあってかつ前後に比較級’cuter’や2つの名詞が挟まっていたりしたら、何々と何々を比較して’より’というような単語になるだろうなどと考えられるわけです。また最後には’cuter’があるから’かわいい’となるというような感じで、人間が英文翻訳をするのと同じような注目の仕方でそれぞれの単語の状態を見て翻訳のタスクを行うのがAttention機構がやっていることのイメージとなります。実際、人間が翻訳するとき、’dogはcat（thanがあるから）より（cuterがあるから）かわいい’というような注目の仕方で行っていると思います。

次の図は非常に複雑ですが、どういう風に入力文の状態を注目して出力を出すかという点については1つに限定された定義が存在するわけではなくいろんな実装の仕方があります。今回はTransformerというモデルで使われているもので、内積注意と呼ばれていてる行列の内積という演算を使って各ベクトルの関連の度合いを計算するものです。

図の文章の場合には、’He loves to eat.’という文章がありますが、こちらをRNNにどんどん入力していきます。それぞれのRNNの中間層の状態は１つのベクトルになりますが、それらのベクトルと出力部分の状態の内積を取ることで2つのベクトルの関連性を調べます。ちなみに、内積というのは、掛け合わせるベクトルが同じ方向を向いていれば１で、直交になると0になり、真逆の方向を向いていれば-1になるようなそのような値を取る計算式なので、内積の大きさというものが2つのベクトルの間の関連度合い、つまりどれくらい似ているのかを表すのに用いることができます。この内積の計算をすることで、入力の状態と出力の状態の一番似ているところを図の赤の矢印のようにSoftmax関数に取り出して入力のところに掛けてあげることで、入力と出力の一番近いところを取ってきてあげるというができるようになります。

今の例では、入力と出力の関連性を見ました。次の図のものはもう少し複雑な説明にはなりますが、１つのベクトルに対して同じ計算を行っていくと、同じ文章の中でどの単語とどの単語が関連し合っているかを学習することができるようになります。この技術はSelf-Attention（自己注意）と呼ばれています。こちらの図はもともとミクシーの方によって書かれた書かれたものを利用させていただいていますが、たとえば、好きな動物はというような文章があったときに、このSelf-Attentionを用いて学習すると、図の〇の大きさが関連性の高さを示しているのですが、’動物’という単語と’好き’という単語の関連性が高いというような学習結果が得られます。’動物’という名詞がどういう動詞に関連性があるか、つまり単語がどの単語を修飾しているかといったことがSelf-Attentionを使うと判定できます。

次の図は、TransformerというGoogleのニューラル機械翻訳のモデルの論文から取ってきたものになりますが、’The law will never … ‘というような文章があった場合に、図の’its’が何を指しているのかを学習することができます。’its’は代名詞なので、その前にすでに具体的な単語があってそれを指し示しているのですが、それはこの場合は’law’とつながっているとか、’its application’の’its’が何を修飾しているかというようなことを確認することができます。つまり、このSelf-Attentionというものが文章の文脈の表現を抽出するのに用いることが可能であるということをここで示しています。再度説明すると、文章つながりであったり、単語同士の関係であったりといったことを抽出することがSelf-Attentionを使うことで可能になります。

ここまでの説明でようやくBERTがどういうものかということが何となくイメージできるようになったのではないかと思います。これからBERTというものをもう少し説明してきます。

BERTの詳細説明

下図がBERTのモデルの構造になりますが、いままで説明したSelf-Attention機構を使って文章の文脈を獲得する部分が下図の右側の図のモデルの赤の太枠で表示されている部分にあたります。下図ではtmとして示されているこのSelf-Attention機構を図の左枠の部分のようにたくさん並べます。まず文章は時系列で左側から順番に流れるようになっていますが、それだけではなくそれぞれのSelf-Attention機構には図の左枠内の矢印のように右から左向きに文章の流れとは逆方向の接続も設けることにより、さらに強力に表現を抽出できるようにした双方向の言語表現をエンコードするモデルとなっています。

BERTを使うと何がうれしいかというと、文章の表現を学習および記述することができることです。何らかの日本語の文章で、たとえば、’神田の鬼金棒のラーメンは美味しい’のような文章が入ってきた場合に、画像の表現抽出器のように、下図の大規模データセットで訓練済みのBERTとして示されているような中間の表現抽出器を使ってある中間状態にして、それを下図のように利用することで、機械語翻訳モデル、対話モデルやこの文章がポジティブかネガティブかを調べるような感情推定モデルなどを作ったり、いろんなタスクに応用できたりします。ありがたいことに、京都大学などでWikipediaなど大規模な文章のデータセットを使って、BERTのモデルの訓練を行ってくれています。Googleが提供しているモデルには、英語、中国語など多言語対応しているものもあれば、日本語に対応したものもあります。いろんな人がやりたいようなタスクの性能をあげるためのひとつのマリオのスパワーアップキノコのような存在としてBERTを使うことができます。

以下の図はBERTが多数のベンチマークタスクの性能のリストで圧倒的な高い性能を発揮したということで話題になりました。State-Of-The-ArtとはAIのベンチマークの世界で最高性能のことを指します。たとえばSWAGというベンチマークテストはどういったものかというと、’A girl is going across a set up monkey bars. She’のような文章があった場合、それに続く文章を当てるというようなタスクになっています。選択肢としては図に示されるように4つのものがありますが、この中で一番自然なつながりがどれなのかを選ぶというものです。これはなかなか人間でも少し時間をかけて考えることで理解できるような難しい問題設定ですが、このようなものもBERTを使うとかなり高い精度で正解することができます。このような部分がBERTがすごくて驚くべきところです。

では、BERT自体はどのように学習されているかというと、Wikipediaのような文章を入力すると勝手に学習するわけではなくて、BERTに対して何かしらのテストというか宿題のようなものを用意してあげる必要があります。やっていることは下図に示す穴埋めクイズと隣接文クイズというものを解くことで、それらをBERTに学習をさせています。これはどういうものかというと、’the men went to grocery store’ というような文章があって、図のようにその中の１つの単語、この場合はgroceryの部分をマスクすることで、中にはどのような単語があったでしょうかというようなことを学ばせることをやっていたり、さきほどのSWAGのタスクと似た感じで、入力文章から次の文章がどれかを当てさせたりするようなものです。図の例の場合では、上の文章が正解なのですけれども、‘男がスーパーに行って1ガロンのミルクを買いました’というような文章になるため、文章として適切であると言えるので、ラベルにIsNexttというものが付いています。それ以外に図の下の文章のようにランダムに別のところから文章を取ってきますが、たいていランダムにとってきた場合には前後の文章としては意味が成立しないので、ラベルとしてはNotNextというものが付いてくるわけですが、実際2番目の文章には“ペンギンは鳥より飛ばない”というように隣接する文章として正しいか否かを当てるようなタスクもあります。このようなことを学習していくことで、文脈にあたる表現を獲得できるようになっていきます。

このようにして獲得したBERTのモデルに対していろいろな出力のユニットがありますが、それにアドオンでネットワークを追加してあげれば、さまざまな機能を実現でき多様なタスクに応用できるようになります。たとえば、下図の赤の点のように、分類タスク用のネットワークを追加すれば、分類ができるようになるし、文章を出力するようなネットワークを後ろに追加してあげれば翻訳などができるようになります。

まとめ

まとめとしては、まず時系列データに対しては、全結合ニューラルネットワークよりリカーレントニューラルネットワークの方が向いています。またリカーレントネットワークもゲートなどを追加することで単純なリカーレントニューラルネットワークの欠点を解消するこができるようになります。リカーレントネットワークの有名なモデルとしては、LSTMやGRUなどあります。機械翻訳みたいなタスクに対しては、Seq2seqのモデルが良く使われますが、その入力の文脈情報を１つのベクトルとしてまとめるのではなくて、Attentionを使ってどの文脈に注目するかを時間の関数にするというのも近年行われています。特にSelf-Attention機構はリカーレントニューラルネットワークを使っていなくても強力な文脈表現として用いることができるものです。またBERTというのは、大量のデータセットを使って事前に学習しておいてあげれば、いろんなタスクに応用できる自然言語処理のドーピング剤のような存在であります。
